In [72]:
import csv
import itertools
import sys

PROBS = {

    # Unconditional probabilities for having gene
    "gene": {
        2: 0.01,
        1: 0.03,
        0: 0.96
    },

    "trait": {

        # Probability of trait given two copies of gene
        2: {
            True: 0.65,
            False: 0.35
        },

        # Probability of trait given one copy of gene
        1: {
            True: 0.56,
            False: 0.44
        },

        # Probability of trait given no gene
        0: {
            True: 0.01,
            False: 0.99
        }
    },

    # Mutation probability
    "mutation": 0.01
}


def main():

    # Check for proper usage
    #if len(sys.argv) != 2:
    #    sys.exit("Usage: python heredity.py data.csv")
    #people = load_data(sys.argv[1])

    people = load_data("data/family0.csv")

    # Keep track of gene and trait probabilities for each person
    probabilities = {
        person: {
            "gene": {
                2: 0,
                1: 0,
                0: 0
            },
            "trait": {
                True: 0,
                False: 0
            }
        }
        for person in people
    }

    # Loop over all sets of people who might have the trait
    names = set(people)
    for have_trait in powerset(names):

        # Check if current set of people violates known information
        fails_evidence = any(
            (people[person]["trait"] is not None and
             people[person]["trait"] != (person in have_trait))
            for person in names
        )
        if fails_evidence:
            continue

        # Loop over all sets of people who might have the gene
        for one_gene in powerset(names):
            for two_genes in powerset(names - one_gene):

                # Update probabilities with new joint probability
                p = joint_probability(people, one_gene, two_genes, have_trait)
                update(probabilities, one_gene, two_genes, have_trait, p)

    # Ensure probabilities sum to 1
    normalize(probabilities)

    # Print results
    for person in people:
        print(f"{person}:")
        for field in probabilities[person]:
            print(f"  {field.capitalize()}:")
            for value in probabilities[person][field]:
                p = probabilities[person][field][value]
                print(f"    {value}: {p:.4f}")


def load_data(filename):
    """
    Load gene and trait data from a file into a dictionary.
    File assumed to be a CSV containing fields name, mother, father, trait.
    mother, father must both be blank, or both be valid names in the CSV.
    trait should be 0 or 1 if trait is known, blank otherwise.
    """
    data = dict()
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row["name"]
            data[name] = {
                "name": name,
                "mother": row["mother"] or None,
                "father": row["father"] or None,
                "trait": (True if row["trait"] == "1" else
                          False if row["trait"] == "0" else None)
            }
    return data


def powerset(s):
    """
    Return a list of all possible subsets of set s.
    """
    s = list(s)
    return [
        set(s) for s in itertools.chain.from_iterable(
            itertools.combinations(s, r) for r in range(len(s) + 1)
        )
    ]


def joint_probability(people, one_gene, two_genes, have_trait):
    """
    Compute and return a joint probability.

    The probability returned should be the probability that
        * everyone in set `one_gene` has one copy of the gene, and
        * everyone in set `two_genes` has two copies of the gene, and
        * everyone not in `one_gene` or `two_gene` does not have the gene, and
        * everyone in set `have_trait` has the trait, and
        * everyone not in set` have_trait` does not have the trait.
    """

    # CG: initialize probability:
    joint_prob  = 1

    # CG: iterate thru all people:
    for person in people:
        # CG: get individual number of genes:
        num_genes = count_genes(person, one_gene, two_genes)

        # CG: compute probability given the number of genes:
        joint_prob = joint_prob * PROBS["gene"][num_genes]

        # CG: add parents's inherited probabilities:
        if people[person]["mother"] != None:
            joint_prob = joint_prob * parents_gene_prob(people[person], one_gene, two_genes)

        # CG: check if a person has the trait...:
        has_trait = person in have_trait

        # CG: adds probability if a person has the trait or not:
        joint_prob = joint_prob * PROBS["trait"][num_genes][has_trait]

    # CG: returns the result:
    return joint_prob


# CG: helper to compute number of genes of a person:
def count_genes(person, one_gene, two_genes):
    # CG: initialize return value (0=no genes):
    result = 0
    # CG: return 1 if person has 1 gene:
    if person in one_gene:
        result = 1
    # CG: return 2 if person has 2 genes:
    elif person in two_genes:
        result = 2
    # CG: return the result:
    return result


# CG: helper to compute parents combined inherited probability:
def parents_gene_prob(person, one_gene, two_genes):
    # CG: initialize result:
    result = 1

    # CG: get number of mother's genes:
    num_genes = count_genes(person["mother"], one_gene, two_genes)

    # CG: if mom has 0 genes, prob of mutation is minimum:
    if num_genes == 0:
        mom = PROBS["mutation"] 
    # CG: if mom has 1 gene, she passes the gene with 50% prob of no mutation:
    elif num_genes == 1:
        mom = (1 - PROBS["mutation"]) / 2 
    # CG: if mom has 2 genes, prob of mutation is maximum:
    elif num_genes == 2:
        mom = (1 - PROBS["mutation"])

    # CG: get no. of father's genes:
    num_genes = count_genes(person["father"], one_gene, two_genes)

    # CG: if dad has 0 genes, prob of mutation is minimum:
    if num_genes == 0:
        dad = PROBS["mutation"]
    # CG: if dad has 1 gene, he passes the gene with 50% prob of no mutation:
    elif num_genes == 1:
        dad = (1 - PROBS["mutation"]) / 2 
    # CG: if dad has 2 genes, prob of mutation is maximum:
    elif num_genes == 2:
        dad = (1 - PROBS["mutation"])

    # CG: compute parents' combined probability:
    result = mom * dad

    # CG: return the result:
    return result


def update(probabilities, one_gene, two_genes, have_trait, p):
    """
    Add to `probabilities` a new joint probability `p`.
    Each person should have their "gene" and "trait" distributions updated.
    Which value for each distribution is updated depends on whether
    the person is in `have_gene` and `have_trait`, respectively.
    """
    # CG: iterate thru all people's probabilities:
    for person in probabilities:
        # CG: get individual number of genes:
        num_genes = count_genes(person, one_gene, two_genes)

        # CG: add 'p' to the probability given the number of genes:
        probabilities[person]['gene'][num_genes] += p

        # CG: check if a person has the trait...:
        has_trait = person in have_trait

        # CG: ...and add the probability accordingly
        probabilities[person]['trait'][has_trait] += p


def normalize(probabilities):
    """
    Update `probabilities` such that each probability distribution
    is normalized (i.e., sums to 1, with relative proportions the same).
    """
    for person in probabilities:
        # CG: compute total sum of probabilities to have a gene for a given person:
        sum_probs = sum(list(probabilities[person]['gene'].values()))

        # CG: compute and update with proportion:
        for i in range (3):
            probabilities[person]["gene"][i] = probabilities[person]["gene"][i] / sum_probs

        # CG: compute total sum of probabilities to have trait for a given person:
        sum_trait = sum(list(probabilities[person]["trait"].values()))

        for i in list ([True, False]):
            probabilities[person]["trait"][i] = probabilities[person]["trait"][i] / sum_trait


if __name__ == "__main__":
    main()


Harry:
  Gene:
    2: 0.0100
    1: 0.0300
    0: 0.9600
  Trait:
    True: 0.0329
    False: 0.9671
James:
  Gene:
    2: 0.4334
    1: 0.5601
    0: 0.0065
  Trait:
    True: 1.0000
    False: 0.0000
Lily:
  Gene:
    2: 0.1777
    1: 0.3350
    0: 0.4873
  Trait:
    True: 0.0000
    False: 1.0000
